**CLASIFICACIÓN DE CACAO POR ETAPAS DE MADURACIÓN**

Este proyecto tiene como objetivo clasificar imágenes de cacao, de acuerdo a sus etapas de maduración. Se presentan en 4 diferentes etapas; la primera de cero-dos meses; la segunda de dos-cuatro meses; la tercera de cuatro-seis meses; la cuarta de seis meses en adelante. 
Esto cobra importancia en diversos sectores como la agricultura y la investigación. Dado que se pude tener un mayor control y una planeacipon de los cultivos de cacao y sirve para el estudio de los mismos.

Se obtuvieron las imágenes del siguiente repositorio: https://doi.org/10.5281/zenodo.7968315, las imágenes están en formato COCO, el data set contiene 4116 imágenes, que se encuentran clasificadas en cinco tipos. Cada tipo de imagen se encuentra etiquetada, cada una de estas etiquetas en archivos independientes, de estos el tipo A no será utilizado en este proyecto, debido a que contiene imágenes no relevantes para el problema. 

In [ ]:
# Se instalan librerías
%pip install pycocotools 
%pip install scikit-learn

Se crea el data frame llamado df, el cual contiene la información de las imágenes. Posteriormente se divide en tres conjuntos; test, validación y entrenamiento. Se dividieron de la siguiente manera; Train: 2795 imágenes; Validación: 597 imágenes y Test: 597 imágenes. Se balancearon las imágenes para que cada clase contenga aproximadamente la misma cantidad, usando operaciones como rotación, translación, etc. Por último, el data frame queda conformado con la siguiente información: image_id (clave primaria), file_name (nombre del archivo), path (ruta), width y height (medidas), categories, class_name y class_number.

In [7]:
import os
import json
import pandas as pd
import numpy as np
from PIL import Image

def load_custom_dataset(base_path, folder_names):
    """
    Carga datasets COCO desde la estructura específica de carpetas.
    
    Parameters:
    base_path (str): Ruta base (RipSetCocoaCNCH12/RipSetCocoaCNCH12/COCO_10)
    folder_names (list): Lista de nombres de carpetas (images_C1, images_C2, etc.)
    
    Returns:
    pandas.DataFrame: DataFrame consolidado con toda la información
    """
    all_image_data = []
    
    # Imprimir las rutas para debug
    print(f"Ruta base: {base_path}")
    print("Carpetas a procesar:", folder_names)
    
    for folder_name in folder_names:
        # Construir rutas
        images_path = os.path.join(base_path, folder_name)
        annotation_path = os.path.join(base_path, f'{folder_name}.json')
        
        print(f"\nProcesando carpeta: {folder_name}")
        print(f"Ruta de imágenes: {images_path}")
        print(f"Ruta de anotaciones: {annotation_path}")
        
        try:
            # Verificar si el archivo JSON existe
            if not os.path.exists(annotation_path):
                print(f"¡Advertencia! No se encontró el archivo: {annotation_path}")
                continue
                
            # Cargar archivo JSON
            with open(annotation_path, 'r') as f:
                coco_data = json.load(f)
            
            print(f"JSON cargado exitosamente para {folder_name}")
            
            # Crear diccionario de categorías
            categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
            
            # Crear diccionario de imágenes
            images = {img['id']: img for img in coco_data['images']}
            
            # Organizar anotaciones por imagen
            image_annotations = {}
            for ann in coco_data['annotations']:
                img_id = ann['image_id']
                if img_id not in image_annotations:
                    image_annotations[img_id] = set()
                image_annotations[img_id].add(categories[ann['category_id']])
            
            # Obtener el número de clase del nombre de la carpeta
            class_number = int(folder_name.split('_C')[1])
            
            # Preparar datos para el DataFrame
            for img_id, annotations in image_annotations.items():
                img_info = images[img_id]
                image_entry = {
                    'image_id': img_id,
                    'file_name': img_info['file_name'],
                    'path': os.path.join(images_path, img_info['file_name']),
                    'width': img_info.get('width', None),
                    'height': img_info.get('height', None),
                    'categories': list(annotations),
                    'class_name': folder_name,  # Guardamos el nombre de la carpeta
                    'class_number': class_number  # Guardamos el número de clase
                }
                all_image_data.append(image_entry)
            
            print(f"Procesadas {len(image_annotations)} imágenes de {folder_name}")
                
        except Exception as e:
            print(f"Error procesando {folder_name}: {str(e)}")
    
    # Crear DataFrame consolidado
    if not all_image_data:
        raise ValueError("No se encontraron datos en ninguna carpeta")
    
    df = pd.DataFrame(all_image_data)
    return df


In [8]:
def create_train_val_test_split(df, val_size=0.15, test_size=0.15, seed=42):
    """
    Crea splits estratificados manteniendo la distribución de clases.
    """
    np.random.seed(seed)
    
    train_data, val_data, test_data = [], [], []
    
    # Procesar por clase
    for class_name in df['class_name'].unique():
        class_data = df[df['class_name'] == class_name]
        
        # Calcular tamaños
        n_samples = len(class_data)
        n_test = int(n_samples * test_size)
        n_val = int(n_samples * val_size)
        
        # Dividir los datos
        shuffled = class_data.sample(frac=1, random_state=seed)
        train_data.append(shuffled[:(n_samples-n_test-n_val)])
        val_data.append(shuffled[(n_samples-n_test-n_val):(n_samples-n_test)])
        test_data.append(shuffled[(n_samples-n_test):])
    
    # Combinar los datos
    train_df = pd.concat(train_data).reset_index(drop=True)
    val_df = pd.concat(val_data).reset_index(drop=True)
    test_df = pd.concat(test_data).reset_index(drop=True)
    
    return train_df, val_df, test_df

def verify_dataset(df):
    """
    Verifica la integridad del dataset.
    """
    verification = {
        'total_images': len(df),
        'images_per_class': df['class_name'].value_counts().to_dict(),
        'missing_images': [],
        'categories_per_class': {}
    }
    
    # Verificar que las imágenes existen
    for idx, row in df.iterrows():
        if not os.path.exists(row['path']):
            verification['missing_images'].append(row['path'])
    
    # Analizar categorías por clase
    for class_name in df['class_name'].unique():
        class_data = df[df['class_name'] == class_name]
        categories = [cat for cats in class_data['categories'] for cat in cats]
        verification['categories_per_class'][class_name] = pd.Series(categories).value_counts().to_dict()
    
    return verification

In [ ]:
# 1. Definir las rutas
ruta_1 = 'C:\\Users\\melis\\Desktop\\Proyecto_Cocoa\\RipSetCocoaCNCH12\\RipSetCocoaCNCH12\\COCO_10'
ruta_2 = ['images_C1', 'images_C2', 'images_C3', 'images_C4']

# 2. Cargar el dataset
try:
    df = load_custom_dataset(ruta_1, ruta_2)
    
    # 3. Verificar la integridad del dataset
    verification = verify_dataset(df)
    print("\nEstadísticas del dataset:")
    print(f"Total de imágenes: {verification['total_images']}")
    print("\nImágenes por clase:")
    for class_name, count in verification['images_per_class'].items():
        print(f"{class_name}: {count}")

    if verification['missing_images']:
        print("\n¡Advertencia! Imágenes faltantes:", len(verification['missing_images']))

    # 4. Crear los splits
    train_df, val_df, test_df = create_train_val_test_split(df)

    # 5. Ver distribución final
    print("\nDistribución final:")
    print(f"Train: {len(train_df)} imágenes")
    print(f"Validación: {len(val_df)} imágenes")
    print(f"Test: {len(test_df)} imágenes")

except Exception as e:
    print(f"Error en el proceso: {str(e)}")

In [ ]:
train_df.head()

Se toman 10 imágenes de manera aleatoria y se fijan para posteriormente realizar muestras con ellas.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

# Muestra aleatoria de 10 imágenes
sample_images = train_df.sample(n=10, random_state=42)

# Mostrar las imágenes seleccionadas
plt.figure(figsize=(15, 10))  # Tamaño del lienzo

for i, row in enumerate(sample_images.iterrows()):
    img_path = row[1]['path']  # Ruta de la imagen
    img = Image.open(img_path)  # Abrir la imagen
    
    plt.subplot(2, 5, i + 1)  # Crear subgráficos (2 filas, 5 columnas)
    plt.imshow(img)  # Mostrar la imagen
    plt.axis('off')  # Ocultar ejes
    plt.title(row[1]['class_name'])  # Título con la categoría de la imagen

plt.tight_layout()
plt.show()


A continuación, se hacen exploraciones como canny, espacios de color (HSV, LAB y RGB).

In [ ]:
import cv2


import matplotlib.pyplot as plt
from PIL import Image

import numpy as np

import matplotlib.colors as mcolors

def plot(imagen_1, transform_func, transform_name="Transformación", **kwargs):
    """
    Muestra imágenes originales y procesadas con cualquier función de transformación.
    
    Args:
    - imagen_1: DataFrame que contiene las rutas de las imágenes.
    - transform_func: Función de transformación que se aplicará a cada imagen.
                      Debe aceptar un parámetro (ruta de la imagen) y devolver la imagen transformada.
    - transform_name: Nombre de la transformación, para usar en los títulos (por defecto, "Transformación").
    - **kwargs: Argumentos adicionales que serán pasados a la función de transformación.
    
    Returns:
    - None: Visualización directa de las imágenes.
    """
    plt.figure(figsize=(20, 50))  # Incrementar el tamaño del lienzo

    for i, row in enumerate(imagen_1.iterrows()):
        img_path = row[1]['path']  # Ruta de la imagen
        
        # Imagen original
        img = Image.open(img_path)
        plt.subplot(10, 2, 2 * i + 1)  # Una fila por imagen, columna 1 para original
        plt.imshow(img)
        plt.axis('off')
        plt.title('Original', fontsize=16)
        
        # Imagen procesada
        transformed_img = transform_func(img_path, **kwargs)
        plt.subplot(10, 2, 2 * i + 2)
        
        if len(transformed_img.shape) == 2:  # Canal individual
            color_space = kwargs.get('color_space', None)
            channel = kwargs.get('channel', None)
            
            if color_space == "HSV" and channel in [1, 2]:
                # Escalar canal S o V de HSV para visualizarlo como intensidad
                color_image = cv2.cvtColor(transformed_img, cv2.COLOR_GRAY2RGB)
                plt.imshow(color_image)
            elif color_space == "HSV" and channel == 0:
                # Canal Hue en colores
                plt.imshow(transformed_img, cmap='hsv')
            elif color_space == "Lab":
                if channel == 0:
                    # Canal L (Luminosidad) como escala de grises
                    plt.imshow(transformed_img, cmap='gray')
                elif channel == 1:
                    # Canal a: verde a rojo
                    plt.imshow(transformed_img, cmap=mcolors.LinearSegmentedColormap.from_list('green-red', ['green', 'gray', 'red']))
                elif channel == 2:
                    # Canal b: azul a amarillo
                    plt.imshow(transformed_img, cmap=mcolors.LinearSegmentedColormap.from_list('blue-yellow', ['blue', 'gray', 'yellow']))
            else:
                # Escala de grises por defecto
                plt.imshow(transformed_img, cmap='gray')
        else:  # Imagen completa (RGB o similar)
            plt.imshow(transformed_img)
        
        plt.axis('off')
        plt.title(transform_name, fontsize=16)

    plt.tight_layout(pad=5.0)  # Aumentar el espacio entre las imágenes
    plt.show()




def apply_canny_edge_detection(image_path, threshold1=50, threshold2=100):
    """
    Aplica la detección de bordes tipo Canny a una imagen.
    
    Args:
    - image_path: Ruta de la imagen.
    - threshold1: Umbral inferior para la detección de bordes.
    - threshold2: Umbral superior para la detección de bordes.
    
    Returns:
    - edge_image: Imagen con los bordes detectados.
    """
    # Cargar la imagen en escala de grises
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Aplicar la detección de bordes
    edges = cv2.Canny(img, threshold1, threshold2)
    
    return edges

muestra=train_df.sample(n=10, random_state=42)



plot(muestra,apply_canny_edge_detection,"canny")



In [13]:
def transform_color_space(image_path, color_space="RGB", channel=None):
    """
    Transforma la imagen a un espacio de color específico y permite extraer un canal.
    
    Args:
    - image_path: Ruta de la imagen.
    - color_space: Espacio de color al que se transformará la imagen.
                   Valores soportados: "RGB", "HSV", "Lab", "BGR".
    - channel: Canal específico a extraer (0, 1, 2). Por ejemplo, "0" para el canal L en Lab.
    
    Returns:
    - transformed_img: Imagen transformada o canal específico.
    """
    # Leer la imagen
    img = cv2.imread(image_path)
    
    # Transformar al espacio de color especificado
    if color_space == "RGB":
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    elif color_space == "GRAY":
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    elif color_space == "HSV":
        img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    elif color_space == "Lab":
        img = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
    elif color_space == "BGR":
        pass  # Sin cambios
    else:
        raise ValueError(f"Espacio de color '{color_space}' no soportado.")
    
    # Extraer un canal específico si se indica
    if channel is not None:
        if len(img.shape) == 3:  # Asegurarse de que la imagen tiene canales
            img = img[:, :, channel]
        else:
            raise ValueError(f"La imagen no tiene canales disponibles para extraer.")

    return img





In [ ]:
muestra_1=muestra

plot(
    imagen_1=muestra_1,
    transform_func=transform_color_space,
    transform_name="Espacio de color HSV",
    color_space="HSV",
    
)



In [ ]:
muestra_2=muestra



plot(
    imagen_1=muestra_2,
    transform_func=transform_color_space,
    transform_name="Espacio de color LAB",
    color_space="Lab"
    
)



In [ ]:
muestra_3=muestra
plot(
    imagen_1=muestra_3,
    transform_func=transform_color_space,
    transform_name="Espacio de color RGB",
    color_space="RGB"

)






Se presentan los histogramas de las imágenes originales

In [17]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def plot_histogram(muestra, transform_func=None, color_space="RGB", **kwargs):
    """
    Muestra los histogramas de las imágenes en la muestra.
    
    Args:
    - muestra: DataFrame con las rutas de las imágenes.
    - transform_func: Función de transformación para cambiar el espacio de color (opcional).
                      Si None, se analiza el espacio de color original.
    - color_space: Espacio de color para las imágenes (por defecto, "RGB").
                   Otros valores: "GRAY", "HSV", "Lab".
    - **kwargs: Argumentos adicionales que serán pasados a la función de transformación.
    
    Returns:
    - None: Visualización directa de los histogramas.
    """
    for i, row in enumerate(muestra.iterrows()):
        img_path = row[1]['path']  # Ruta de la imagen
        img = cv2.imread(img_path)  # Leer imagen en BGR
        
        # Transformar espacio de color si se especifica
        if transform_func:
            img = transform_func(img_path, **kwargs)
        
        # Espacios de color
        if color_space == "RGB":
            channels = ['Red', 'Green', 'Blue']
            colors = ['r', 'g', 'b']
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        elif color_space == "HSV":
            channels = ['Hue', 'Saturation', 'Value']
            colors = ['m', 'c', 'y']
            img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        elif color_space == "Lab":
            channels = ['L', 'a', 'b']
            colors = ['gray', 'g', 'b']
            img = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
        elif color_space == "GRAY":
            channels = ['Gray']
            colors = ['k']
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        else:
            raise ValueError(f"Espacio de color '{color_space}' no soportado.")
        
        # Mostrar histogramas
        plt.figure(figsize=(10, 6))
        plt.title(f"Histogram for Image {i+1}")
        
        if len(img.shape) == 2:  # Escala de grises
            hist = cv2.calcHist([img], [0], None, [256], [0, 256])
            plt.plot(hist, color='k', label='Gray')
        else:  # Canales de color
            for j, (channel, color) in enumerate(zip(channels, colors)):
                hist = cv2.calcHist([img], [j], None, [256], [0, 256])
                plt.plot(hist, color=color, label=channel)
        
        plt.xlabel("Intensity Value")
        plt.ylabel("Frequency")
        plt.legend()
        plt.show()


In [ ]:
plot_histogram(
    muestra=muestra
)

Se aplica un filtro de suavisado gaussiano y CLAHE a las imágenes, obteniendo buenos resultados en la calidad de estas.

In [19]:
import cv2
import matplotlib.pyplot as plt

def preprocess_image_to_display(image_path, clip_limit=2.0, tile_grid_size=(8, 8), kernel_size=(5, 5)):
    """
    Aplica un filtro de suavizado gaussiano y CLAHE a una imagen y la devuelve para visualización.
    
    Args:
    - image_path: Ruta de la imagen de entrada.
    - clip_limit: Parámetro de contraste para CLAHE.
    - tile_grid_size: Tamaño de las regiones para CLAHE.
    - kernel_size: Tamaño del kernel para el filtro gaussiano.
    
    Returns:
    - processed_img: Imagen procesada.
    """
    # Leer la imagen
    img = cv2.imread(image_path)
    
    # Aplicar filtro de suavizado gaussiano
    blurred_img = cv2.GaussianBlur(img, kernel_size, 0)
    
    # Convertir a Lab para aplicar CLAHE
    lab = cv2.cvtColor(blurred_img, cv2.COLOR_BGR2Lab)
    l, a, b = cv2.split(lab)
    
    # Aplicar CLAHE al canal L
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    cl = clahe.apply(l)
    
    # Reconstruir la imagen Lab y convertir a BGR
    lab = cv2.merge((cl, a, b))
    processed_img = cv2.cvtColor(lab, cv2.COLOR_Lab2BGR)
    
    return processed_img


def display_processed_images(df, path_column='path', clip_limit=2.0, tile_grid_size=(8, 8), kernel_size=(5, 5)):
    """
    Procesa un conjunto de imágenes desde un DataFrame y las muestra en pantalla.
    
    Args:
    - df: DataFrame con las rutas de las imágenes.
    - path_column: Nombre de la columna en el DataFrame que contiene las rutas de las imágenes.
    - clip_limit: Parámetro de contraste para CLAHE.
    - tile_grid_size: Tamaño de las regiones para CLAHE.
    - kernel_size: Tamaño del kernel para el filtro gaussiano.
    """
    for _, row in df.iterrows():
        image_path = row[path_column]
        
        # Leer y procesar la imagen
        original_img = cv2.imread(image_path)
        original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)  # Convertir a RGB para matplotlib
        processed_img = preprocess_image_to_display(
            image_path,
            clip_limit=clip_limit,
            tile_grid_size=tile_grid_size,
            kernel_size=kernel_size
        )
        processed_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB)  # Convertir a RGB para matplotlib
        
        # Mostrar la imagen original y la procesada
        plt.figure(figsize=(10, 5))
        
        # Imagen original
        plt.subplot(1, 2, 1)
        plt.imshow(original_img)
        plt.title("Original")
        plt.axis("off")
        
        # Imagen procesada
        plt.subplot(1, 2, 2)
        plt.imshow(processed_img)
        plt.title("Procesada (Suavizado + CLAHE)")
        plt.axis("off")
        
        plt.show()


In [ ]:
display_processed_images(
    df=muestra,
    path_column='path',  # Cambia si tu columna tiene otro nombre.
    clip_limit=2.0, 
    tile_grid_size=(8, 8), 
    kernel_size=(5, 5)
)


Se hace una comparación de la imagen original con su histograma y de la imagen procesada también con su respectivo histograma.

In [21]:
import cv2
import matplotlib.pyplot as plt

def preprocess_image_with_histogram(image_path, clip_limit=2.0, tile_grid_size=(8, 8), kernel_size=(5, 5)):
    """
    Aplica un filtro de suavizado gaussiano y CLAHE a una imagen y devuelve la imagen procesada y sus histogramas.
    
    Args:
    - image_path: Ruta de la imagen de entrada.
    - clip_limit: Parámetro de contraste para CLAHE.
    - tile_grid_size: Tamaño de las regiones para CLAHE.
    - kernel_size: Tamaño del kernel para el filtro gaussiano.
    
    Returns:
    - processed_img: Imagen procesada.
    """
    # Leer la imagen
    img = cv2.imread(image_path)
    
    # Aplicar filtro de suavizado gaussiano
    blurred_img = cv2.GaussianBlur(img, kernel_size, 0)
    
    # Convertir a Lab para aplicar CLAHE
    lab = cv2.cvtColor(blurred_img, cv2.COLOR_BGR2Lab)
    l, a, b = cv2.split(lab)
    
    # Aplicar CLAHE al canal L
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    cl = clahe.apply(l)
    
    # Reconstruir la imagen Lab y convertir a BGR
    lab = cv2.merge((cl, a, b))
    processed_img = cv2.cvtColor(lab, cv2.COLOR_Lab2BGR)
    
    return processed_img


def display_images_and_histograms(df, path_column='path', clip_limit=2.0, tile_grid_size=(8, 8), kernel_size=(5, 5)):
    """
    Procesa un conjunto de imágenes desde un DataFrame, muestra las imágenes y sus histogramas.
    
    Args:
    - df: DataFrame con las rutas de las imágenes.
    - path_column: Nombre de la columna en el DataFrame que contiene las rutas de las imágenes.
    - clip_limit: Parámetro de contraste para CLAHE.
    - tile_grid_size: Tamaño de las regiones para CLAHE.
    - kernel_size: Tamaño del kernel para el filtro gaussiano.
    """
    for _, row in df.iterrows():
        image_path = row[path_column]
        
        # Leer la imagen original
        original_img = cv2.imread(image_path)
        original_img_rgb = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)  # Convertir a RGB para matplotlib
        
        # Procesar la imagen
        processed_img = preprocess_image_with_histogram(
            image_path,
            clip_limit=clip_limit,
            tile_grid_size=tile_grid_size,
            kernel_size=kernel_size
        )
        processed_img_rgb = cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB)  # Convertir a RGB para matplotlib
        
        # Calcular histogramas
        def calculate_histogram(image, color_space='RGB'):
            histograms = {}
            if color_space == 'RGB':
                channels = ['Red', 'Green', 'Blue']
                colors = ['r', 'g', 'b']
                for i, (channel, color) in enumerate(zip(channels, colors)):
                    hist = cv2.calcHist([image], [i], None, [256], [0, 256])
                    histograms[channel] = (hist, color)
            return histograms
        
        original_histograms = calculate_histogram(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
        processed_histograms = calculate_histogram(cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB))
        
        # Mostrar imágenes y histogramas
        plt.figure(figsize=(20, 10))
        
        # Imagen original
        plt.subplot(2, 4, 1)
        plt.imshow(original_img_rgb)
        plt.title("Original")
        plt.axis("off")
        
        # Histograma de la imagen original
        plt.subplot(2, 4, 2)
        for channel, (hist, color) in original_histograms.items():
            plt.plot(hist, color=color, label=channel)
        plt.title("Histograma Original")
        plt.legend()
        
        # Imagen procesada
        plt.subplot(2, 4, 3)
        plt.imshow(processed_img_rgb)
        plt.title("Procesada (Suavizado + CLAHE)")
        plt.axis("off")
        
        # Histograma de la imagen procesada
        plt.subplot(2, 4, 4)
        for channel, (hist, color) in processed_histograms.items():
            plt.plot(hist, color=color, label=channel)
        plt.title("Histograma Procesado")
        plt.legend()
        
        plt.tight_layout()
        plt.show()


In [ ]:
display_images_and_histograms(
    df=muestra,
    path_column='path',  # Cambia si tu columna tiene otro nombre.
    clip_limit=2.0, 
    tile_grid_size=(8, 8), 
    kernel_size=(5, 5)
)


Se aplica normalización y estandarizacion a la imagen original y se muestras las tres imágenes.

In [23]:
def normalize_image(image):
    """
    Normaliza la imagen para que los valores de los píxeles estén en el rango [0, 1].
    
    Args:
    - image: Imagen original (array de NumPy).
    
    Returns:
    - normalized_img: Imagen normalizada (array de NumPy).
    """
    return image / 255.0  # Escala los valores a [0, 1]

def normalize_dataset(df, path_column='path'):
    """
    Aplica normalización a todas las imágenes de un DataFrame.
    
    Args:
    - df: DataFrame con las rutas de las imágenes.
    - path_column: Nombre de la columna que contiene las rutas de las imágenes.
    
    Returns:
    - normalized_images: Lista de imágenes normalizadas.
    """
    normalized_images = []
    
    for _, row in df.iterrows():
        image_path = row[path_column]
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convertir a RGB
        normalized_img = normalize_image(img)
        normalized_images.append(normalized_img)
    
    return normalized_images


In [24]:
def standardize_image(image):
    """
    Estandariza la imagen para que tenga media 0 y desviación estándar 1.
    
    Args:
    - image: Imagen original (array de NumPy).
    
    Returns:
    - standardized_img: Imagen estandarizada (array de NumPy).
    """
    mean = np.mean(image, axis=(0, 1), keepdims=True)
    std = np.std(image, axis=(0, 1), keepdims=True)
    standardized_img = (image - mean) / (std + 1e-8)  # Evita divisiones por cero
    return standardized_img

def standardize_dataset(df, path_column='path'):
    """
    Aplica estandarización a todas las imágenes de un DataFrame.
    
    Args:
    - df: DataFrame con las rutas de las imágenes.
    - path_column: Nombre de la columna que contiene las rutas de las imágenes.
    
    Returns:
    - standardized_images: Lista de imágenes estandarizadas.
    """
    standardized_images = []
    
    for _, row in df.iterrows():
        image_path = row[path_column]
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convertir a RGB
        standardized_img = standardize_image(img)
        standardized_images.append(standardized_img)
    
    return standardized_images


In [ ]:
normalized_images = normalize_dataset(df=muestra, path_column='path')
print(f"Total de imágenes normalizadas: {len(normalized_images)}")


In [ ]:
standardized_images = standardize_dataset(df=muestra, path_column='path')
print(f"Total de imágenes estandarizadas: {len(standardized_images)}")


In [27]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

def display_normalized_and_standardized(df, path_column='path'):
    """
    Muestra imágenes originales, normalizadas y estandarizadas lado a lado.
    
    Args:
    - df: DataFrame con las rutas de las imágenes.
    - path_column: Nombre de la columna que contiene las rutas de las imágenes.
    """
    for _, row in df.iterrows():
        image_path = row[path_column]
        
        # Leer la imagen original
        original_img = cv2.imread(image_path)
        original_img_rgb = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)  # Convertir a RGB para matplotlib
        
        # Normalización
        normalized_img = original_img_rgb / 255.0
        
        # Estandarización
        mean = np.mean(original_img_rgb, axis=(0, 1), keepdims=True)
        std = np.std(original_img_rgb, axis=(0, 1), keepdims=True)
        standardized_img = (original_img_rgb - mean) / (std + 1e-8)
        standardized_img = np.clip((standardized_img - standardized_img.min()) / (standardized_img.max() - standardized_img.min()), 0, 1)  # Reescala para visualización
        
        # Mostrar las imágenes
        plt.figure(figsize=(15, 5))
        
        # Imagen original
        plt.subplot(1, 3, 1)
        plt.imshow(original_img_rgb)
        plt.title("Original")
        plt.axis("off")
        
        # Imagen normalizada
        plt.subplot(1, 3, 2)
        plt.imshow(normalized_img)
        plt.title("Normalizada (0 a 1)")
        plt.axis("off")
        
        # Imagen estandarizada
        plt.subplot(1, 3, 3)
        plt.imshow(standardized_img)
        plt.title("Estandarizada (Media=0, Var=1)")
        plt.axis("off")
        
        plt.show()


In [ ]:
display_normalized_and_standardized(df=muestra, path_column='path')


Se hace una comparación de la imagen procesada (suavizada y CLAHE), la normalizada y la estandarizada.

In [29]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

def preprocess_image(image_path, clip_limit=2.0, tile_grid_size=(8, 8), kernel_size=(5, 5)):
    """
    Aplica suavizado Gaussiano y CLAHE a una imagen.
    
    Args:
    - image_path: Ruta de la imagen original.
    - clip_limit: Parámetro de contraste para CLAHE.
    - tile_grid_size: Tamaño de las regiones para CLAHE.
    - kernel_size: Tamaño del kernel para el filtro Gaussiano.
    
    Returns:
    - processed_img: Imagen procesada.
    """
    # Leer la imagen
    img = cv2.imread(image_path)
    
    # Aplicar filtro de suavizado gaussiano
    blurred_img = cv2.GaussianBlur(img, kernel_size, 0)
    
    # Convertir a Lab para aplicar CLAHE
    lab = cv2.cvtColor(blurred_img, cv2.COLOR_BGR2Lab)
    l, a, b = cv2.split(lab)
    
    # Aplicar CLAHE al canal L
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    cl = clahe.apply(l)
    
    # Reconstruir la imagen Lab y convertir a BGR
    lab = cv2.merge((cl, a, b))
    processed_img = cv2.cvtColor(lab, cv2.COLOR_Lab2BGR)
    
    return processed_img


def normalize_image(image):
    """
    Normaliza una imagen para que los valores estén en el rango [0, 1].
    
    Args:
    - image: Imagen procesada (array de NumPy).
    
    Returns:
    - normalized_img: Imagen normalizada (array de NumPy).
    """
    return image / 255.0  # Escala los valores a [0, 1]


def standardize_image(image):
    """
    Estandariza una imagen para que tenga media 0 y desviación estándar 1.
    
    Args:
    - image: Imagen procesada (array de NumPy).
    
    Returns:
    - standardized_img: Imagen estandarizada (array de NumPy).
    """
    mean = np.mean(image, axis=(0, 1), keepdims=True)
    std = np.std(image, axis=(0, 1), keepdims=True)
    standardized_img = (image - mean) / (std + 1e-8)  # Evitar divisiones por cero
    return standardized_img


def process_and_display_images(df, path_column='path', clip_limit=2.0, tile_grid_size=(8, 8), kernel_size=(5, 5)):
    """
    Aplica suavizado y CLAHE, luego normaliza y estandariza las imágenes, y las muestra en pantalla.
    
    Args:
    - df: DataFrame con las rutas de las imágenes.
    - path_column: Nombre de la columna que contiene las rutas de las imágenes.
    - clip_limit: Parámetro de contraste para CLAHE.
    - tile_grid_size: Tamaño de las regiones para CLAHE.
    - kernel_size: Tamaño del kernel para el filtro Gaussiano.
    """
    for _, row in df.iterrows():
        image_path = row[path_column]
        
        # Procesar la imagen (suavizado + CLAHE)
        processed_img = preprocess_image(
            image_path,
            clip_limit=clip_limit,
            tile_grid_size=tile_grid_size,
            kernel_size=kernel_size
        )
        processed_img_rgb = cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB)  # Convertir a RGB para matplotlib
        
        # Normalizar la imagen procesada
        normalized_img = normalize_image(processed_img_rgb)
        
        # Estandarizar la imagen procesada
        standardized_img = standardize_image(processed_img_rgb)
        standardized_img = np.clip((standardized_img - standardized_img.min()) / (standardized_img.max() - standardized_img.min()), 0, 1)  # Reescala para visualización
        
        # Mostrar las imágenes
        plt.figure(figsize=(20, 6))
        
        # Imagen procesada
        plt.subplot(1, 3, 1)
        plt.imshow(processed_img_rgb)
        plt.title("Procesada (Suavizado + CLAHE)")
        plt.axis("off")
        
        # Imagen normalizada
        plt.subplot(1, 3, 2)
        plt.imshow(normalized_img)
        plt.title("Normalizada (0 a 1)")
        plt.axis("off")
        
        # Imagen estandarizada
        plt.subplot(1, 3, 3)
        plt.imshow(standardized_img)
        plt.title("Estandarizada (Media=0, Var=1)")
        plt.axis("off")
        
        plt.show()


In [ ]:
process_and_display_images(
    df=muestra,
    path_column='path',  # Cambia si tu columna tiene otro nombre.
    clip_limit=2.0, 
    tile_grid_size=(8, 8), 
    kernel_size=(5, 5)
)


Se realizan varias transformaciones no lineales en las imágenes, todas se muestran a continuación.

In [31]:
def adjust_gamma(image, gamma=1.0):
    image = image.astype(np.float32)
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(256)]).astype("uint8")
    return cv2.LUT(image.astype(np.uint8), table)

def apply_gamma_to_muestra(df, path_column='path', gamma=1.0):
    for _, row in df.iterrows():
        image_path = row[path_column]
        img = cv2.imread(image_path)
        original_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gamma_img = adjust_gamma(img, gamma=gamma)
        gamma_img_rgb = cv2.cvtColor(gamma_img, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(15, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(original_img)
        plt.title("Original")
        plt.axis("off")
        plt.subplot(1, 2, 2)
        plt.imshow(gamma_img_rgb)
        plt.title(f"Transformada Gamma (Gamma={gamma})")
        plt.axis("off")
        plt.show()


In [32]:
def apply_log_transform(image):
    image = image.astype(np.float32)
    c = 255 / np.log(1 + np.max(image))
    log_image = c * np.log(1 + image)
    return np.uint8(log_image)

def apply_log_to_muestra(df, path_column='path'):
    for _, row in df.iterrows():
        image_path = row[path_column]
        img = cv2.imread(image_path)
        original_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        log_img = apply_log_transform(img)
        log_img_rgb = cv2.cvtColor(log_img, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(15, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(original_img)
        plt.title("Original")
        plt.axis("off")
        plt.subplot(1, 2, 2)
        plt.imshow(log_img_rgb)
        plt.title("Transformada Logarítmica")
        plt.axis("off")
        plt.show()


In [33]:
def apply_exponential_transform(image):
    image = image.astype(np.float32) / 255.0
    exp_image = np.exp(image) - 1
    exp_image = np.clip(exp_image / np.max(exp_image), 0, 1) * 255
    return np.uint8(exp_image)

def apply_exponential_to_muestra(df, path_column='path'):
    for _, row in df.iterrows():
        image_path = row[path_column]
        img = cv2.imread(image_path)
        original_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        exp_img = apply_exponential_transform(img)
        exp_img_rgb = cv2.cvtColor(exp_img, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(15, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(original_img)
        plt.title("Original")
        plt.axis("off")
        plt.subplot(1, 2, 2)
        plt.imshow(exp_img_rgb)
        plt.title("Transformada Exponencial")
        plt.axis("off")
        plt.show()



In [34]:
def apply_sigmoid_transform(image, gain=5, cutoff=128):
    image = image.astype(np.float32)
    sigmoid_image = 1 / (1 + np.exp(-gain * (image - cutoff) / 255.0))
    return (sigmoid_image * 255).astype(np.uint8)

def apply_sigmoid_to_muestra(df, path_column='path', gain=5, cutoff=128):
    for _, row in df.iterrows():
        image_path = row[path_column]
        img = cv2.imread(image_path)
        original_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        sigmoid_img = apply_sigmoid_transform(img, gain=gain, cutoff=cutoff)
        sigmoid_img_rgb = cv2.cvtColor(sigmoid_img, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(15, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(original_img)
        plt.title("Original")
        plt.axis("off")
        plt.subplot(1, 2, 2)
        plt.imshow(sigmoid_img_rgb)
        plt.title(f"Transformada Sigmoidal (Gain={gain}, Cutoff={cutoff})")
        plt.axis("off")
        plt.show()


In [35]:
def apply_bilateral_filter(image, diameter=15, sigma_color=75, sigma_space=75):
    return cv2.bilateralFilter(image, diameter, sigma_color, sigma_space)

def apply_bilateral_to_muestra(df, path_column='path', diameter=15, sigma_color=75, sigma_space=75):
    for _, row in df.iterrows():
        image_path = row[path_column]
        img = cv2.imread(image_path)
        original_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        bilateral_img = apply_bilateral_filter(img, diameter, sigma_color, sigma_space)
        bilateral_img_rgb = cv2.cvtColor(bilateral_img, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(15, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(original_img)
        plt.title("Original")
        plt.axis("off")
        plt.subplot(1, 2, 2)
        plt.imshow(bilateral_img_rgb)
        plt.title("Filtro Bilateral")
        plt.axis("off")
        plt.show()


In [36]:
def apply_adaptive_threshold(image, method=cv2.ADAPTIVE_THRESH_GAUSSIAN_C):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.adaptiveThreshold(gray, 255, method, cv2.THRESH_BINARY, 11, 2)

def apply_adaptive_threshold_to_muestra(df, path_column='path', method=cv2.ADAPTIVE_THRESH_GAUSSIAN_C):
    for _, row in df.iterrows():
        image_path = row[path_column]
        img = cv2.imread(image_path)
        original_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        thresh_img = apply_adaptive_threshold(img, method=method)
        plt.figure(figsize=(15, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(original_img)
        plt.title("Original")
        plt.axis("off")
        plt.subplot(1, 2, 2)
        plt.imshow(thresh_img, cmap='gray')
        plt.title("Umbral Adaptativo")
        plt.axis("off")
        plt.show()


In [ ]:
apply_gamma_to_muestra(muestra, path_column='path', gamma=1.2)
apply_log_to_muestra(muestra, path_column='path')
apply_exponential_to_muestra(muestra, path_column='path')
apply_sigmoid_to_muestra(muestra, path_column='path', gain=5, cutoff=128)
apply_bilateral_to_muestra(muestra, path_column='path')
apply_adaptive_threshold_to_muestra(muestra, path_column='path')


Operaciones morfologicas
